### Initialization

In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")

from typing import List

import numpy as np
from matplotlib import pyplot as plt
import open3d as o3d
import pandas as pd
import cv2 as cv



%matplotlib inline
# plt.rcParams["figure.figsize"] = [20, 10]

# import the ZOD DevKit
from zod import ZodSequences

# NOTE! Set the path to dataset and choose a version
folder_path = "/Users/quany/Downloads/GPDF_MPC_ADAS"  # local folder 
dataset_root = folder_path+"/mini"  # your local path to zod
version = "full"  # "mini" or "full"

# initialize ZodSequences
zod_sequences = ZodSequences(dataset_root=dataset_root, version=version)

# Fetch certain zod sequence
seq = zod_sequences[330]

/Users/quany/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Loading infos: 0it [00:00, ?it/s]

### Log tracking results

In [6]:
import pickle

# Load tracking result from the file
file_path = folder_path+"/log_obs3_file.pkl"

with open(file_path, 'rb') as file:
    log_obs3 = pickle.load(file)

# Read lidar timestamps
T_lidar = []
frames = seq.info.get_lidar_frames()
for i, frame in enumerate(frames):
    lidar_frame = frame
    pcd = seq.get_compensated_lidar(lidar_frame.time)
    pcd_timestamp = pcd.timestamps[0]
    T_lidar.append(pcd_timestamp)

# Filter out the tracking result with empty detection and the corresponding lidar frame
obs_filtered = []
T_lidar_filtered = []
for i, log in enumerate(log_obs3):
    if len(log) > 0:
        obs_filtered.append(log)
        T_lidar_filtered.append(T_lidar[i])



### Log OxTS data

In [7]:
# Read oxts data with the closest timestamps in the T_oxts for each timestamp in T_lidar
# X_oxts = seq.oxts.poses[:,0]
# Y_oxts = seq.oxts.poses[:,1]
V_oxts = seq.oxts.velocities[:,0]
A_oxts = seq.oxts.accelerations[:,0]
T_oxts = seq.oxts.timestamps

# Interpolate the oxts data to the timestamps in T_lidar
# X_ego = []
# Y_ego = []
V_ego = []
A_ego = []
for t in T_lidar_filtered:
    idx = np.argmin(np.abs(T_oxts - t))
    # X_ego.append(X_oxts[idx])
    # Y_ego.append(Y_oxts[idx])
    V_ego.append(V_oxts[idx])
    A_ego.append(A_oxts[idx])

# integrate V_lon_closest to get the distance with repect to timestamps in T_lidar
X_ego = np.zeros(len(V_ego))
Y_ego = np.zeros(len(V_ego))
for i in range(1, len(V_ego)):
    X_ego[i] = X_ego[i-1] + V_ego[i] * (T_lidar[i] - T_lidar[i-1])


### Project tracking result to global frame

In [8]:
# Extract the local tracking results from log_obs3
obs_pos = []
for i, x in enumerate(obs_filtered):
    if x == []:
        pos = []
    else:
        pos = (X_ego[i]+x[0][3] - x[0][5],Y_ego[i]-(x[0][2] + 0.5 * x[0][4]))
    obs_pos.append(pos)


### Log to .mat file

In [9]:
import scipy.io as sio

# Save the tracking results and ego vehicle velocity to a file for MATLAB
file_path = folder_path+"/log330.mat"
sio.savemat(file_path, {
    'obs_pos': obs_pos,
    'A_ego': A_ego,
    'V_ego': V_ego,
    'X_ego': X_ego,
    'Y_ego': Y_ego,
    'obs_timestamp': T_lidar_filtered
})
print("Data has been saved to the MATLAB file")

Data has been saved to the MATLAB file
